In [ ]:
import gdown
url = 'https://drive.google.com/u/0/uc?id=1KtFphaabPpN5hrTEYVwrDMmyR1tKXpKN&export=download'
output = 'data.zip'
gdown.download(url, output, quiet=False)
gdown.extractall('data.zip')

Downloading...
From: https://drive.google.com/u/0/uc?id=1KtFphaabPpN5hrTEYVwrDMmyR1tKXpKN&export=download
To: /content/data.zip
100%|██████████| 394M/394M [00:06<00:00, 56.7MB/s]


['s2/',
 's2/pwad3p.mpg',
 's2/lgir8s.mpg',
 's2/lwbz3n.mpg',
 's2/sbbuzs.mpg',
 's2/prac7p.mpg',
 's2/sgwj6a.mpg',
 's2/sgib8s.mpg',
 's2/lbax7n.mpg',
 's2/prii6s.mpg',
 's2/lgaz5n.mpg',
 's2/pwij2s.mpg',
 's2/pbbi7n.mpg',
 's2/srau4a.mpg',
 's2/pbwx1p.mpg',
 's2/pbbv6s.mpg',
 's2/bbbz8s.mpg',
 's2/lbiqzs.mpg',
 's2/swwc4s.mpg',
 's2/bgan5p.mpg',
 's2/lrbr4s.mpg',
 's2/pgix6s.mpg',
 's2/brbm7p.mpg',
 's2/bgin1p.mpg',
 's2/bwbt7p.mpg',
 's2/srbb5p.mpg',
 's2/sgib7n.mpg',
 's2/sbat5n.mpg',
 's2/bwwn7p.mpg',
 's2/sria6s.mpg',
 's2/sgbv9p.mpg',
 's2/pwbd5n.mpg',
 's2/lbwe6a.mpg',
 's2/pwwy3p.mpg',
 's2/prwd3n.mpg',
 's2/pgaq5n.mpg',
 's2/pwwk6a.mpg',
 's2/swao4s.mpg',
 's2/lbid1n.mpg',
 's2/lrae1n.mpg',
 's2/pbapza.mpg',
 's2/pbib7p.mpg',
 's2/prwq1n.mpg',
 's2/bgah1p.mpg',
 's2/lriq8a.mpg',
 's2/lgaz8a.mpg',
 's2/swau7n.mpg',
 's2/pgid5p.mpg',
 's2/pgij7n.mpg',
 's2/srwvzs.mpg',
 's2/lrby8s.mpg',
 's2/pwij4a.mpg',
 's2/sbwu6a.mpg',
 's2/sgbv7n.mpg',
 's2/prbc9n.mpg',
 's2/pgiq1n.mpg',
 '

In [ ]:
# import gdown
# url = 'https://drive.google.com/file/d/1I2-NMZmwT9hvEr-QAn1TWt_4UB8tK7u9/view?usp=drive_link'
# output = 'shape_pred_file.zip'
# gdown.download(url, output, quiet=False)
# gdown.extractall('shape_pred_file.zip')

In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import dlib
import imageio

In [ ]:
# face_detector = dlib.get_frontal_face_detector()
# landmark_predictor = dlib.shape_predictor("/content/dlib_shape_pred/test/shape_predictor_68_face_landmarks.dat")
# def load_video(path):

#     cap = cv2.VideoCapture(path)
#     frames = []
#     for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
#         ret, frame = cap.read()
#         frame = tf.image.rgb_to_grayscale(frame).numpy()
#         face = face_detector(frame)
#         if len(face)>0:
#             landmarks = landmark_predictor(frame, face[0])
#             lip_landmarks = landmarks.parts()[48:68]
#             lip_landmarks_array = np.array([(p.x, p.y) for p in lip_landmarks])

#             x, y, w, h = cv2.boundingRect(lip_landmarks_array)
#             lip_image = frame[y:y + h, x:x + w, :]
#             res = cv2.resize(lip_image, dsize=(100, 50), interpolation=cv2.INTER_CUBIC)[..., None]
#             frames.append(res)


#     frames = np.array(frames).astype(np.float32)
#     mean = np.mean(frames)
#     std = np.std(frames)
#     return (frames-mean) / std

In [ ]:
def load_video(path):

    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        frames.append(frame[190:236,80:220,:])
    cap.release()

    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames - mean), tf.float32) / std

In [ ]:
load_video("/content/data/s1/bbaf2n.mpg").shape

TensorShape([75, 46, 140, 1])

In [ ]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz "]

In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

In [ ]:
def load_alignments(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens = [*tokens,' ',line[2]]
    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [ ]:
def load_data(path: str):
    path = bytes.decode(path.numpy())
    file_name = os.path.basename(path).split('.')[0]
    # print(file_name)
    video_path = os.path.join('data','s1',f'{file_name}.mpg')
    alignment_path = os.path.join("content","data", "alignments", "s1", f"{file_name}.align")
    # print(video_path, alignment_path)
    frames = load_video(video_path)
    alignments = load_alignments(f"/{alignment_path}")

    return frames, alignments

In [ ]:
def map_function(path):
    result = tf.py_function(load_data, [path], (tf.float32, tf.int64))
    return result

In [ ]:
data = tf.data.Dataset.list_files('/content/data/s1/*.mpg')
data = data.shuffle(500, reshuffle_each_iteration=False)
data = data.map(map_function)
data = data.padded_batch(2, padded_shapes=([75,None,None,None],[40]))
data = data.prefetch(tf.data.AUTOTUNE)
# Added for split
train = data.take(450)
test = data.skip(450)

In [ ]:
frames, alignment =data.as_numpy_iterator().next()

In [ ]:
frames

array([[[[[1.4424336 ],
          [1.4424336 ],
          [1.4803925 ],
          ...,
          [0.        ],
          [0.03795878],
          [0.03795878]],

         [[1.4424336 ],
          [1.4424336 ],
          [1.4803925 ],
          ...,
          [0.        ],
          [0.03795878],
          [0.03795878]],

         [[1.3665161 ],
          [1.3665161 ],
          [1.3665161 ],
          ...,
          [0.03795878],
          [0.03795878],
          [0.03795878]],

         ...,

         [[0.9869283 ],
          [0.9869283 ],
          [0.9489695 ],
          ...,
          [9.489695  ],
          [9.489695  ],
          [9.489695  ]],

         [[0.9869283 ],
          [0.9869283 ],
          [0.9489695 ],
          ...,
          [9.489695  ],
          [9.489695  ],
          [9.489695  ]],

         [[0.9869283 ],
          [0.9869283 ],
          [0.9489695 ],
          ...,
          [9.489695  ],
          [9.489695  ],
          [9.489695  ]]],


        [[[1.4424

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, GRU, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback

In [ ]:
model = Sequential()
model.add(Conv3D(128, 3, input_shape=(75,46,140,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(256, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(75, 3, padding='same'))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(GRU(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Bidirectional(GRU(128, kernel_initializer='Orthogonal', return_sequences=True)))
model.add(Dropout(.5))

model.add(Dense(char_to_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 75, 46, 140, 128   3584      
                             )                                   
                                                                 
 activation (Activation)     (None, 75, 46, 140, 128   0         
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 75, 23, 70, 128)   0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 75, 23, 70, 256)   884992    
                                                                 
 activation_1 (Activation)   (None, 75, 23, 70, 256)   0         
                                                        

In [ ]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:
checkpoint_callback = ModelCheckpoint('/content/models/checkpoint', monitor = "loss", save_weights_only=True)

In [ ]:
# class TestCallback(Callback):

#   def __init__(self, dataset):
#     self.dataset = dataset.as_numpy_iterator()

#   def on_epoch_end(self, epoch, logs=None):
#     frames, alignment = self.dataset.next()
#     result = self.model.predict(frames)
#     result = tf.argmax(result[0], axis=1)
#     predicted_result = tf.strings.reduce_join(num_to_char(tf.argmax(result, axis=1))).numpy().decode("utf-8")
#     original_alignment = tf.strings.reduce_join(num_to_char(tf.argmax(alignment[0], axis=1))).numpy().decode("utf-8")
#     print(f'ORIGINAL :: {original_alignment}')
#     print(f'PREDICTED :: {predicted_result}')

class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset) -> None:
        self.dataset = dataset.as_numpy_iterator()

    def on_epoch_end(self, epoch, logs=None) -> None:
        data = self.dataset.next()
        result= self.model.predict(data[0])
        decoded = tf.keras.backend.ctc_decode(result, [75,75], greedy=False)[0][0].numpy()
        for x in range(len(result)):
            print('Original:', tf.strings.reduce_join(num_to_char(data[1][x])).numpy().decode('utf-8'))
            print('Prediction:',

                  )
            print('~'*100)

In [ ]:
test_callback = TestCallback(test)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=CTCLoss)

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1ZjV3BqqMk95gZLgy126P0lw8b-93guTj&export=download'
output = 'pre_models.zip'
gdown.download(url, output, quiet=False)
gdown.extractall('pre_models.zip')

Downloading...
From: https://drive.google.com/u/0/uc?id=1ZjV3BqqMk95gZLgy126P0lw8b-93guTj&export=download
To: /content/pre_models.zip
100%|██████████| 75.4M/75.4M [00:00<00:00, 182MB/s]


['models/.ipynb_checkpoints/',
 'models/checkpoint',
 'models/checkpoint.data-00000-of-00001',
 'models/checkpoint.index']

In [ ]:
model.load_weights("/content/models/checkpoint")

In [ ]:
model.fit(train, validation_data=test, epochs=100, callbacks=[checkpoint_callback, test_callback])

Epoch 1/100
1/1 [==============================] - 0s 236ms/step
Original: set green in i four please
Prediction: set gren in i four please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Original: lay green in f two please
Prediction: lay gren in f two please
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
450/450 [==============================] - 675s 2s/step - loss: 7.0132 - val_loss: 3.0215
Epoch 2/100
450/450 [==============================] - ETA: 0s - loss: 6.6873

UnknownError: ignored

In [ ]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("dlib_shape_pred/shape_predictor_68_face_landmarks.dat")
def load_video(path):

    cap = cv2.VideoCapture(path)
    frames = []
    file_name = os.path.basename(path).split('.')[0]
    dir_name = f"frames/{file_name}"
    os.makedirs(dir_name, exist_ok=True)
    print("created dir: "+ dir_name)

    for _ in range(min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),75)):
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame).numpy()
        face = face_detector(frame)
        if len(face)>0:
            landmarks = landmark_predictor(frame, face[0])
            lip_landmarks = landmarks.parts()[48:68]
            lip_landmarks_array = np.array([(p.x, p.y) for p in lip_landmarks])

            x, y, w, h = cv2.boundingRect(lip_landmarks_array)
            lip_image = frame[y:y + h, x:x + w, :]
            res = cv2.resize(lip_image, dsize=(100, 50), interpolation=cv2.INTER_CUBIC)[..., None]
            done = cv2.imwrite(f"frames/{file_name}/{_}.jpg", res)
            if(not done):
                print("Error")

In [ ]:
!zip -r /content/models_12_49.zip /content/models

  adding: content/models/ (stored 0%)
  adding: content/models/.ipynb_checkpoints/ (stored 0%)
  adding: content/models/checkpoint.data-00000-of-00001 (deflated 6%)
  adding: content/models/checkpoint.index (deflated 68%)
  adding: content/models/checkpoint (deflated 48%)


In [ ]:
from google.colab import files
files.download("/content/models_12_49.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>